In [1]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import mean_squared_error

In [2]:
#Join iata data
iata = pd.read_csv('data/IATA_US_CODES.csv')
iata['IATA Code']=iata['IATA Code'].astype(str)
iata['IATA Code']=iata['IATA Code'].str.rstrip()
iata=iata.set_index('IATA Code')
iata_dep=iata.add_suffix('_Dep')
iata_arr=iata.add_suffix('_Arr')
iata['2018']=iata['2018'].str.replace(',','')
iata['2017']=iata['2017'].str.replace(',','')
iata['2016']=iata['2016'].str.replace(',','')
iata['2015']=iata['2015'].str.replace(',','')
iata['2018']=iata['2018'].str.replace('\xa0','')
iata['2017']=iata['2017'].str.replace('\xa0','')
iata['2016']=iata['2016'].str.replace('\xa0','')
iata['2015']=iata['2015'].str.replace('\xa0','')
iata['State']=iata['State'].str.replace('\xa0','')
iata['Major city served']=iata['Major city served'].str.replace('New York/','')
iata['Major city served']=iata['Major city served'].str.replace('/St. Paul','')
iata['Major city served']=iata['Major city served'].str.replace('Baltimore/','')
iata['Major city served']=iata['Major city served'].str.replace('/Tacoma','')
iata['Major city served']=iata['Major city served'].str.replace('/Fort Worth','')
iata['Major city served']=iata['Major city served'].str.replace(', D.C.','')
iata

,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015
IATA Code,,,,,,,,
ATL,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
LAX,2,Los Angeles International Airport,Los Angeles,CA,42626783,41232432,39636042,36351226
ORD,3,O'Hare International Airport,Chicago,IL,39874879,38593028,37589899,36305668
DFW,4,Dallas/Fort Worth International Airport,Dallas,TX,32800721,31816933,31283579,31589832
DEN,5,Denver International Airport,Denver,CO,31363573,29809097,28267394,26280043
JFK,6,John F. Kennedy International Airport,New York,NY,30541459,29533154,29239151,27782369
SFO,7,San Francisco International Airport,San Francisco,CA,27794154,26900048,25707101,24190549
SEA,8,Seattle–Tacoma International Airport,Seattle,WA,24894338,22639124,21887110,21231781
LAS,9,McCarran International Airport,Las Vegas,NV,23655285,23364393,22833267,21824231


In [3]:
data_weather = pd.read_csv('submissions/starting_kit_2/external_data.csv')
data_weather=data_weather.join(iata, on='AirPort', how='left')
data_weather['Major city served']=data_weather['Major city served'].str.replace('\xa0','')

data_weather['Major city served'].unique()

array(['Atlanta', 'Chicago', 'Los Angeles', 'Dallas', 'Denver',
       'New York', 'San Francisco', 'Charlotte', 'Las Vegas', 'Phoenix',
       'Houston', 'Miami', 'Orlando', 'Newark', 'Seattle', 'Minneapolis',
       'Detroit', 'Philadelphia', 'Boston'], dtype=object)

In [4]:
us_cities = pd.read_csv('data/uscities.csv')

In [5]:
data_weather=data_weather.merge(us_cities, left_on=['Major city served', 'State'],right_on=['city', 'state_id'], how='left')

In [6]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,1

In [7]:
data_weather=data_weather.drop(['city_ascii','state_id','state_name','county_fips','county_name',
                                'county_fips_all','county_name_all','source','military',
                                'incorporated','timezone','zips','id','Major city served',
                               'Airports (large hubs)'], axis=1)

In [8]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1


In [9]:
from math import *

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [10]:
fuel = pd.read_csv('data/fuelprices.csv')
fuel=fuel.drop('Unnamed: 0',axis=1)
list(fuel['Date'].unique())

['2011-09-01',
 '2011-09-02',
 '2011-09-06',
 '2011-09-07',
 '2011-09-08',
 '2011-09-09',
 '2011-09-12',
 '2011-09-13',
 '2011-09-14',
 '2011-09-15',
 '2011-09-16',
 '2011-09-19',
 '2011-09-20',
 '2011-09-21',
 '2011-09-22',
 '2011-09-23',
 '2011-09-26',
 '2011-09-27',
 '2011-09-28',
 '2011-09-29',
 '2011-09-30',
 '2011-10-03',
 '2011-10-04',
 '2011-10-05',
 '2011-10-06',
 '2011-10-07',
 '2011-10-10',
 '2011-10-11',
 '2011-10-12',
 '2011-10-13',
 '2011-10-14',
 '2011-10-17',
 '2011-10-18',
 '2011-10-19',
 '2011-10-20',
 '2011-10-21',
 '2011-10-24',
 '2011-10-25',
 '2011-10-26',
 '2011-10-27',
 '2011-10-28',
 '2011-10-31',
 '2011-11-01',
 '2011-11-02',
 '2011-11-03',
 '2011-11-04',
 '2011-11-07',
 '2011-11-08',
 '2011-11-09',
 '2011-11-10',
 '2011-11-11',
 '2011-11-14',
 '2011-11-15',
 '2011-11-16',
 '2011-11-17',
 '2011-11-18',
 '2011-11-21',
 '2011-11-22',
 '2011-11-23',
 '2011-11-25',
 '2011-11-28',
 '2011-11-29',
 '2011-11-30',
 '2011-12-01',
 '2011-12-02',
 '2011-12-05',
 '2011-12-

In [11]:
data_weather=data_weather.merge(fuel, left_on='Date', right_on='Date', how='left')

In [12]:
data_weather[data_weather['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
9,2011-09-10,ATL,29,22,14,13,11,7,84,56,28,1019,1016,1015,16,16,16,21,9,32.0,0.00,0,NaN,321,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
10,2011-09-11,ATL,30,23,16,13,11,7,72,49,25,1020,1018,1017,16,16,16,21,6,37.0,0.00,2,NaN,294,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,2013-02-18,LGA,2,-2,-7,-12,-15,-17,51,39,26,1024,1022,1016,16,16,16,50,26,64.0,0.00,1,NaN,278,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11029,2013-02-23,LGA,4,3,2,2,1,-1,92,84,76,1025,1017,1009,16,7,0,34,18,42.0,6.86,8,Rain,53,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11030,2013-02-24,LGA,8,5,2,2,-1,-6,92,73,53,1014,1009,1007,16,14,5,45,22,56.0,0.76,7,Rain-Snow,327,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN


In [13]:
data_weather = data_weather.rename(
        columns={'U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)': 'Fuel_price'})

In [14]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [19]:
data_weather['Fuel_price'].fillna(method='ffill', inplace=True)

In [20]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [18]:
data_weather[data_weather['Fuel_price'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price


In [22]:
data_weather['Fuel_price']=data_weather['Fuel_price'].str.replace(',','').astype(int)

In [23]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3005


In [24]:
#data_weather.to_csv('submissions/starting_kit/external_data.csv')

In [25]:
data_weather['Fuel_price'].unique()

array([3088, 3038, 3043,  305, 3018, 2962, 2938, 2922, 2937, 3019, 3002,
       2946, 2919, 2853, 2827,  286, 2928, 2877, 2894, 2844, 2815, 2838,
       2835, 2916, 2881, 2911, 2889, 2917, 2972, 3056, 2986,  301, 2982,
       3028, 3036, 3025, 3006, 3051, 3011, 2991, 3067, 3114, 3119, 3095,
       3131, 3178, 3158, 3156, 3086, 3052, 2964, 3005, 2936, 2914, 2939,
       3001, 2969, 2968, 2941, 2977, 2943, 2867, 2841, 2824, 2854, 2761,
       2749, 2768, 2739, 2829, 2874,  291, 2885, 2891,  293,  304,   31,
        306, 3102, 3134, 3168, 3136, 3079, 3076, 3093, 3045, 3055,  307,
       3062, 3082, 3068, 3085, 3217, 3212, 3197, 3216,  317,  315, 3165,
       3191, 3215, 3237,  329, 3313,  333, 3294, 3239, 3273, 3205,  322,
       3204, 3236, 3288, 3278, 3298, 3292, 3246, 3309, 3272, 3263, 3235,
       3203, 3269, 3262, 3275, 3228, 3242, 3315, 3302, 3234, 3224, 3167,
        324, 3223, 3187, 3201, 3181, 3194, 3261,  326, 3244, 3153, 3049,
       3048, 3023, 2988, 2952, 2893, 2882, 2918, 28